In [31]:
%pip install --upgrade --quiet faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install --upgrade --quiet  docx2txt langchain_community langchain-openai faiss-cpu python-dotenv

Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for chroma-hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: Could not build wheels for chroma-hnswlib, which is required to install pyproject.toml-based projects


In [26]:
from dotenv import load_dotenv
import os


load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key

In [28]:
from langchain_community.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

C:\Users\ACER\AppData\Local\Temp\ipykernel_7832\847912617.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


In [32]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter

In [34]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("D:\Projects\AI opinion generator\Pollens.docx")
loader1 = Docx2txtLoader("D:\Projects\AI opinion generator\Pollens.docx")

data = loader.load()
data1 = loader1.load()

<>:3: SyntaxWarning: invalid escape sequence '\P'
<>:4: SyntaxWarning: invalid escape sequence '\P'
<>:3: SyntaxWarning: invalid escape sequence '\P'
<>:4: SyntaxWarning: invalid escape sequence '\P'
C:\Users\ACER\AppData\Local\Temp\ipykernel_7832\2973927307.py:3: SyntaxWarning: invalid escape sequence '\P'
  loader = Docx2txtLoader("D:\Projects\AI opinion generator\Pollens.docx")
C:\Users\ACER\AppData\Local\Temp\ipykernel_7832\2973927307.py:4: SyntaxWarning: invalid escape sequence '\P'
  loader1 = Docx2txtLoader("D:\Projects\AI opinion generator\Pollens.docx")


In [43]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data+data1)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever()
print(db.index.ntotal)

14


In [73]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name = "gpt-4o-mini", temperature=0)

In [91]:
from langchain.prompts import ChatPromptTemplate 
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser


template = """ you are an expert allergy specialist. 
        Analyze the test report which is given as the query/Question.
        Given the following Context, generate as many specific questions asking the patient 
        whether they have the corresponding symptoms in the context if the test report/query has the condition "Positive". 

        Question: {question}
        Context: {context}
        
        Answer:"""

prompt = ChatPromptTemplate.from_template(template)

In [92]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    |StrOutputParser()
    
)

In [93]:
loader2 = Docx2txtLoader("D:\Projects\AI opinion generator\MAMTHA A9151.docx")
data2 = loader2.load()
print(data2[0].page_content)

ALLERGY TEST (SPT) REPORT

PATIENT NAME:MAMTHA DEVI

                         

                      AGE: 33 Years

                      SEX: FEMALE 

DATE:              08/01/2023

PATIENT REG.NO:  A9151



 TEST NAME: 

 AERO ALLERGEN PANEL                                                                                                        

Test Code

Antigen

Wheal Diameter Size(mm)

Remarks







HOUSE DUST MITE

1

Dermatophagoides Pteronyssius

         13X9MM

POSITIVE

2

Dermatophagoides Farinae

         16X7MM

POSITIVE

3

Bloomia Tropicalis

         8X6MM

POSITIVE

4

Dermatophagoides Pteronyssius(I)

         11X7MM

POSITIVE

5

Dermatophagoides Farinae(I)

        13X6MM

POSITIVE





MOULDS

11

Alternaria Tenuis         

         7X4MM

POSITIVE

12

Aspergillus Fumigatus

         5X4MM

POSITIVE

   13       Cladosporium Herbarum                  8X7MM                                   POSITIVE





ANIMAL DANDER

21

Cockroach

       6X4MM  

POSITIVE

2

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\ACER\AppData\Local\Temp\ipykernel_7832\2602566552.py:1: SyntaxWarning: invalid escape sequence '\P'
  loader2 = Docx2txtLoader("D:\Projects\AI opinion generator\MAMTHA A9151.docx")


In [97]:
Ques=rag_chain.invoke(data2[0].page_content)

In [98]:
print(Ques)

Based on the allergy test report for Mamtha Devi, here are specific questions to ask her regarding symptoms associated with the positive allergens identified in her test:

### Questions Related to House Dust Mites:
1. Do you experience sneezing or a runny nose when you are in dusty environments?
2. Have you noticed any worsening of your asthma symptoms, especially at home?
3. Do you have itchy or watery eyes, particularly when cleaning or dusting?
4. Have you experienced any skin rashes or eczema that seem to flare up in dusty areas?

### Questions Related to Moulds:
5. Do you have a persistent cough or wheezing, especially in damp or moldy areas?
6. Have you noticed any nasal congestion or sinus pressure when exposed to damp environments?
7. Do you experience itchy or irritated eyes when in places with visible mold or dampness?
8. Have you had any skin reactions after being in contact with moldy surfaces?

### Questions Related to Animal Dander:
9. Do you have any symptoms like sneezi

In [115]:

template1 = Ques+""" you are an expert allergy specialist. 
        Analyze the Questions and provide an opinion on whether the patient really has those conditions or not.
       
        Answers:{answers}
        
        """

prompt1 = ChatPromptTemplate.from_template(template1)

In [116]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001AA9F8D6720>, search_kwargs={})

In [117]:
rag_chain1 = (
    {"answers": RunnablePassthrough()}
    | prompt1
    | llm
    |StrOutputParser()
    
)

In [119]:
opinion= rag_chain1.invoke("""Yes, I often sneeze and get a runny nose when I’m in dusty places.

Yes, my asthma symptoms, like shortness of breath, get worse at home, especially in the evenings.

Yes, I notice my eyes become itchy and watery when I’m dusting or cleaning.

Occasionally, I get red, itchy patches on my skin after being in particularly dusty areas.

Yes, I have a persistent dry cough that worsens in damp or moldy areas.

Yes, I frequently feel sinus pressure and nasal congestion after spending time in damp environments.

Yes, my eyes often feel irritated and itchy in places with visible mold or musty smells.

No, I haven’t had any noticeable skin reactions from mold exposure.

Yes, I get sneezing fits and a stuffy nose when I’m around cats or dogs.

Yes, I’ve had hives a couple of times after touching a dog’s fur.

Yes, my asthma acts up more when I’m near animals or in houses where pets are present.

Yes, I often get itchy eyes and sneezing during the warmer months, especially from March to September.

Yes, I sometimes wheeze or feel short of breath when I spend time in grassy areas.

Yes, I’ve experienced allergic reactions, like sneezing and nasal congestion, after being in parks or fields with tall grass.

Yes, I experience nasal congestion and itchy eyes during the peak season for weed pollen.

No, I haven’t had any skin rashes or hives from being outdoors during weed pollen season.

Yes, my asthma tends to worsen in late summer and early fall when these weeds are flowering.

Yes, I experience sneezing and a runny nose during the pollination season for Prosopis juliflora.

Yes, I often feel short of breath and wheeze when I’m near wooded areas during the pollination season.

Yes, I’ve had allergic reactions to tree pollen in both spring and fall seasons."""
)

In [120]:
print(opinion)

Based on Mamtha Devi's responses to the questions regarding her allergy test results, it appears that she is experiencing symptoms consistent with allergies to the identified allergens. Here’s an analysis of her responses:

### House Dust Mites:
1. **Sneezing and runny nose in dusty places**: Yes
2. **Worsening asthma symptoms at home**: Yes
3. **Itchy or watery eyes when cleaning**: Yes
4. **Skin rashes or eczema in dusty areas**: Occasionally

**Opinion**: Mamtha likely has an allergy to house dust mites, as her symptoms align with common reactions to this allergen.

### Moulds:
5. **Persistent cough in damp areas**: Yes
6. **Nasal congestion in damp environments**: Yes
7. **Itchy or irritated eyes in moldy places**: Yes
8. **Skin reactions from mold exposure**: No

**Opinion**: Mamtha shows significant symptoms related to mold exposure, particularly respiratory and ocular symptoms, indicating a likely allergy to mold.

### Animal Dander:
9. **Symptoms around pets**: Yes
10. **Skin r